```SQL
DROP TABLE playlists;
CREATE TABLE playlists (
    playlist_id serial primary key,
    playlist_name varchar,
    number_of_tracks integer
);

INSERT INTO playlists (playlist_name, number_of_tracks)
VALUES
    ('Chill Vibes', 15),
    ('Workout Hits', 10),
    ('Top 50', 50),
    ('Indie Discoveries', 8),
    ('Jazz Essentials', 20),
    ('Classical Mornings', 12),
    ('Party Anthems', 25),
    ('Acoustic Sessions', 18),
    ('Rock Classics', 30),
    ('Pop Culture', 22);

SELECT * FROM playlists;

DROP TABLE playlist_engagement;
CREATE TABLE playlist_engagement (
    playlist_id INTEGER,
    user_id INTEGER,
    listening_time_minutes FLOAT,
    engagement_date DATE
);

INSERT INTO playlist_engagement (user_id, playlist_id, engagement_date, listening_time_minutes)
VALUES
    (101, 1, '2024-10-05', 45),
    (102, 1, '2024-10-06', 30),
    (103, 2, '2024-10-07', 25),
    (104, 2, '2024-10-08', 20),
    (105, 3, '2024-10-09', 120),
    (106, 3, '2024-10-10', 110),
    (107, 4, '2024-10-11', 15),
    (108, 4, '2024-10-12', 10),
    (109, 5, '2024-10-13', 60),
    (110, 5, '2024-10-14', 55),
    (111, 6, '2024-10-15', 40),
    (112, 6, '2024-10-16', 35),
    (113, 7, '2024-10-17', 75),
    (114, 7, '2024-10-18', 70),
    (115, 8, '2024-10-19', 50),
    (116, 8, '2024-10-20', 45),
    (117, 9, '2024-10-21', 95),
    (118, 9, '2024-10-22', 90),
    (119, 10, '2024-10-23', 65),
    (120, 10, '2024-10-24', 60);
```

In [ ]:
import pandas as pd  
import numpy as np

In [13]:
df_playlist = pd.read_csv('Data/playlists.csv')
df_playlist_engagement = pd.read_csv('Data/playlist_engagement.csv')
#df_playlist
df_playlist.head()

,playlist_id,playlist_name,number_of_tracks
0,1,Chill Vibes,15
1,2,Workout Hits,10
2,3,Top 50,50
3,4,Indie Discoveries,8
4,5,Jazz Essentials,20


In [10]:
df_playlist_engagement.head()

,playlist_id,user_id,listening_time_minutes,engagement_date
0,1,101,45,2024-10-05
1,1,102,30,2024-10-06
2,2,103,25,2024-10-07
3,2,104,20,2024-10-08
4,3,105,120,2024-10-09


# Pregunta 1
### El equipo de Recomendaciones de Amazon Music desea saber qué listas de reproducción (playlists) tienen el menor número de canciones. ¿Podrías encontrar la lista de reproducción con la cantidad mínima de canciones?

In [14]:
playlist_min_songs = df_playlist[df_playlist['number_of_tracks'] == df_playlist['number_of_tracks'].min()]
playlist_min_songs

,playlist_id,playlist_name,number_of_tracks
3,4,Indie Discoveries,8


```SQL
SELECT playlist_name, number_of_tracks
FROM playlists
WHERE number_of_tracks = (SELECT MIN(number_of_tracks) FROM playlists);
```

# Pregunta 2

### Estamos interesados en comprender el nivel de interacción (engagement) de las listas de reproducción. Específicamente, queremos identificar qué lista tiene el tiempo promedio de escucha más bajo por canción. Esto significa calcular el tiempo total de escucha para cada lista en octubre de 2024 y luego normalizarlo (dividirlo) por el número de canciones de esa lista. ¿Podrías indicarnos el nombre de la lista de reproducción que tiene el valor más bajo según este cálculo?

In [42]:
# 1. Asegurar que la columna de fecha sea tipo datetime
df_playlist_engagement['engagement_date'] = pd.to_datetime(df_playlist_engagement['engagement_date'])

# 2. Filtrar solo los datos de Octubre 2024
octubre_data = df_playlist_engagement[
    (df_playlist_engagement['engagement_date'] >= '2024-10-01') & 
    (df_playlist_engagement['engagement_date'] <= '2024-10-31')
]

# 3. Calcular el tiempo total de escucha por playlist_id
total_time = octubre_data.groupby('playlist_id')['listening_time_minutes'].sum().reset_index()

# 4. Unir con el DataFrame de playlists para tener el número de tracks y el nombre
df_merged = pd.merge(total_time, df_playlist, on='playlist_id')

# 5. Calcular la métrica: tiempo total / número de tracks
df_merged['time_per_track'] = df_merged['listening_time_minutes'] / df_merged['number_of_tracks']

# 6. Encontrar la playlist con el valor más bajo
playlist_lowest_engagement = df_merged.sort_values('time_per_track').iloc[0]

print(f"La playlist con menor engagement por track es: {playlist_lowest_engagement['playlist_name']}")
print(f"Valor calculado: {playlist_lowest_engagement['time_per_track']:.2f} min/track")

La playlist con menor engagement por track es: Indie Discoveries
Valor calculado: 3.12 min/track


``` SQL
SELECT
    p.playlist_name,
    SUM(e.listening_time_minutes)/p.number_of_tracks AS avg_per_track
FROM playlists p
JOIN playlist_engagement e on p.playlist_id = e.playlist_id
WHERE e.engagement_date BETWEEN '2024-10-01' AND '2024-10-31'
GROUP BY p.playlist_name, p.number_of_tracks
ORDER BY avg_per_track ASC
LIMIT 1;
```

# Pregunta 3

### Para optimizar nuestras recomendaciones, necesitamos el tiempo promedio mensual de escucha por oyente para cada lista de reproducción en octubre de 2024. Por motivos de legibilidad, por favor redondee hacia abajo el tiempo promedio al número entero más cercano.

In [43]:
# 1. Asegurar formato de fecha y filtrar Octubre 2024
df_playlist_engagement['engagement_date'] = pd.to_datetime(df_playlist_engagement['engagement_date'])
octubre_df = df_playlist_engagement[
    (df_playlist_engagement['engagement_date'] >= '2024-10-01') & 
    (df_playlist_engagement['engagement_date'] <= '2024-10-31')
].copy()

# 2. Agrupar por playlist_id para obtener:
#    - Suma total de minutos
#    - Cantidad de usuarios únicos (nunique)
engagement_stats = octubre_df.groupby('playlist_id').agg({
    'listening_time_minutes': 'sum',
    'user_id': 'nunique'
}).reset_index()

# 3. Renombrar columnas para claridad
engagement_stats.columns = ['playlist_id', 'total_time', 'unique_listeners']

# 4. Calcular el promedio por oyente
engagement_stats['avg_per_listener'] = (
    engagement_stats['total_time'] / engagement_stats['unique_listeners']
)

# 5. Redondear hacia abajo (floor) al entero más cercano
engagement_stats['avg_per_listener'] = np.floor(engagement_stats['avg_per_listener']).astype(int)

# 6. Unir con df_playlist para mostrar el nombre de la playlist
resultado_final = pd.merge(
    engagement_stats[['playlist_id', 'avg_per_listener']], 
    df_playlist[['playlist_id', 'playlist_name']], 
    on='playlist_id'
)

# Mostrar resultado ordenado por el promedio más alto
print(resultado_final[['playlist_name', 'avg_per_listener']].sort_values(by='avg_per_listener', ascending=False))

        playlist_name  avg_per_listener
2              Top 50               115
8       Rock Classics                92
6       Party Anthems                72
9         Pop Culture                62
4     Jazz Essentials                57
7   Acoustic Sessions                47
5  Classical Mornings                37
0         Chill Vibes                37
1        Workout Hits                22
3   Indie Discoveries                12


``` SQL
SELECT
    p.playlist_name,
    FLOOR(SUM(e.listening_time_minutes) / COUNT(DISTINCT e.user_id)) AS avg_listening_time
FROM playlists p
JOIN playlist_engagement e ON p.playlist_id = e.playlist_id
WHERE e.engagement_date >= '2024-10-01' AND e.engagement_date <= '2024-10-31'
GROUP BY p.playlist_id, p.playlist_name;
```